In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
# Loading data
df = pd.read_csv('../input/creditcard.csv')

print("Number of samples: {}".format(len(df)))
print("Number of attributes: {}".format(len(df.columns)))

# Input Data Analysis

In [ ]:
df.describe()

In [ ]:
df.sample(5)

In [ ]:
df.groupby("Class").count()["Time"]

# Model Training

## Logistic Regression

In [ ]:
y = df['Class'].copy()
X = df.copy()
del X['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

c_param = 0.1 #should use cross valid to find this
lr = LogisticRegression(C = c_param, penalty = 'l2', class_weight ='balanced', max_iter =100)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
mat_conf = confusion_matrix(y_test, y_pred)
mat_conf

In [ ]:
print("We have {0} well detected fraud (True positives)".format(mat_conf[1,1]))
print("We have {0} undetected fraud (False negatives)".format(mat_conf[1,0]))
print("We have {0} normal behavior classified as fraud (Flase positives)".format(mat_conf[0,1]))


In [ ]:
lr_weight = LogisticRegression(C = c_param, penalty = 'l2', class_weight ={0:1,1:400}, max_iter =100)
lr_weight.fit(X_train, y_train)

y_pred_weight = lr_weight.predict(X_test)
mat_conf_weight = confusion_matrix(y_test, y_pred_weight)
mat_conf_weight

Reducing the numbre of false positive by introducing the class weight

## Multi Layer Perceptron 

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras import backend as K
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle



In [ ]:
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='best')
    plt.show()

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [ ]:
df.Class.value_counts()

In [ ]:
df.sort_values(by='Class', ascending=False, inplace=True) #easier for stratified sampling
#df_full.drop('Time', axis=1,  inplace = True)
df_sample = df.iloc[:3000,:]
shuffle_df = shuffle(df_sample, random_state=42)
df_sample.Class.value_counts()


In [ ]:
y = shuffle_df['Class'].copy()
X = shuffle_df.copy()
del X['Class']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(x_train)
x_train_trans = scaler.transform(x_train)
x_test_trans = scaler.transform(x_test)

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

model = Sequential()
model.add(Dense(200, input_dim=30, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

sgd_opti = SGD(lr=10, momentum=0.01, decay=0.0, nesterov=False)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                   patience=5, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=0, min_lr=1e-8)

early = EarlyStopping(monitor="val_loss", mode="min", verbose=2,
                      patience=30) # probably needs to be more patient, but kaggle time is limited

callbacks_list = [early, reduceLROnPlat]


model.compile( loss ="binary_crossentropy",  #loss='binary_crossentropy',
              optimizer = 'adam', #optimizer = sgd_opti, #optimizer='rmsprop',
              metrics=['accuracy', precision, recall])

ratio = df.groupby("Class").count()["Time"][0] / df.groupby("Class").count()["Time"][1]

class_weight = {0: 5,
                1: 1}

train_history = model.fit(x_train_trans,y_train,
          epochs=200,
          validation_split=0.2,
          batch_size=512,
          class_weight=class_weight)

In [ ]:
show_train_history(train_history,'acc','val_acc')
show_train_history(train_history,'loss','val_loss')
show_train_history(train_history,'precision','val_precision')
show_train_history(train_history,'recall','val_recall')

score = model.evaluate(x_test_trans, y_test, batch_size=128)

In [ ]:
print(score)

In [ ]:
y_pred = model.predict_classes(x_test_trans)
mat_conf = confusion_matrix(y_test, y_pred.astype(int))
mat_conf

### Evaluate on all dataset

In [ ]:
y = df['Class'].copy()
X = df.copy()
del X['Class']
X_TEST = scaler.transform(X)

score = model.evaluate(X_TEST, y, batch_size=128)
print(score)
Y_PRED = model.predict_classes(X_TEST)
mat_conf_all_dataset = confusion_matrix(y,Y_PRED.astype(int))
mat_conf_all_dataset


## Custom Loss

In [ ]:
def custom_loss(y_true,y_pred):
    bce = K.mean(K.binary_crossentropy(y_true,y_pred),axis=-1)
    return bce

In [ ]:
model.compile( loss ="binary_crossentropy",  #loss='binary_crossentropy',
              optimizer = 'adam', #optimizer = sgd_opti, #optimizer='rmsprop',
              metrics=['accuracy', precision, recall])


train_history = model.fit(x_train_trans,y_train,
          epochs=200,
          validation_split=0.5,
          batch_size=512,
          class_weight=class_weight,
        callbacks=callbacks_list,)

In [ ]:
show_train_history(train_history,'acc','val_acc')
show_train_history(train_history,'loss','val_loss')
show_train_history(train_history,'precision','val_precision')
show_train_history(train_history,'recall','val_recall')

score = model.evaluate(x_test_trans, y_test, batch_size=128)

In [ ]:
y_pred = model.predict_classes(x_test_trans)
mat_conf = confusion_matrix(y_test, y_pred.astype(int))
mat_conf

In [ ]:
y = df['Class'].copy()
X = df.copy()
del X['Class']
X_TEST = scaler.transform(X)

score = model.evaluate(X_TEST, y, batch_size=128)
print(score)
Y_PRED = model.predict_classes(X_TEST)
mat_conf_all_dataset = confusion_matrix(y,Y_PRED.astype(int))
mat_conf_all_dataset